#### Outline
- for each dataset: 
    - load dataset; 
    - for each network: 
        - load network
        - project 1000 test dataset samples
        - save to metric dataframe

In [1]:
# reload packages
%load_ext autoreload
%autoreload 2

### Choose GPU (this may not be needed on your computer)

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=''

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=''


In [3]:
import numpy as np
import pickle
import pandas as pd
import time
from umap import UMAP

In [4]:
from tfumap.umap import tfUMAP
import tensorflow as tf
from sklearn.decomposition import PCA
from openTSNE import TSNE

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [5]:
from tqdm.autonotebook import tqdm

In [6]:
from tfumap.paths import ensure_dir, MODEL_DIR, DATA_DIR

In [7]:
output_dir = MODEL_DIR/'projections' 

In [8]:
projection_speeds = pd.DataFrame(columns = ['method_', 'dimensions', 'dataset', 'speed'])

### macosko2015

In [9]:
dataset = 'macosko2015'
dims = [50]

##### load dataset

In [10]:
from tfumap.paths import ensure_dir, MODEL_DIR, DATA_DIR

#dataset_address = 'http://file.biolab.si/opentsne/macosko_2015.pkl.gz'
# https://opentsne.readthedocs.io/en/latest/examples/01_simple_usage/01_simple_usage.html
# also see https://github.com/berenslab/rna-seq-tsne/blob/master/umi-datasets.ipynb

import gzip
import pickle

with gzip.open(DATA_DIR / 'macosko_2015.pkl.gz', "rb") as f:
    data = pickle.load(f)

x = data["pca_50"]
y = data["CellType1"].astype(str)

print("Data set contains %d samples with %d features" % x.shape)

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=.1, random_state=42)

np.shape(X_train)

n_valid = 10000
X_valid = X_train[-n_valid:]
Y_valid = Y_train[-n_valid:]
X_train = X_train[:-n_valid]
Y_train = Y_train[:-n_valid]

X_train_flat = X_train

from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()

Y_train = enc.fit_transform([[i] for i in Y_train]).flatten()

Data set contains 44808 samples with 50 features


In [11]:
X_train_flat = X_train
X_test_flat = X_test

In [12]:
X_train_flat.shape

(30327, 50)

In [13]:
X_test_flat.shape

(4481, 50)

#### Network 

##### 2 dims

In [14]:
load_loc = output_dir / dataset / 'network' 

In [15]:
embedder = tfUMAP(
    direct_embedding=False,
    verbose=True,
    negative_sample_rate=5,
    training_epochs=5,
    batch_size = 100,
    dims = dims
)

In [16]:
encoder = tf.keras.models.load_model((load_loc / 'encoder').as_posix())
embedder.encoder = encoder

In [17]:
n_repeats = 10
times = []
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    embedder.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    times.append(end_time - start_time)
    projection_speeds.loc[len(projection_speeds)] = ['network', 2, dataset, end_time - start_time]


seconds:  0.15242197387851775
seconds:  0.06260958802886307
seconds:  0.06105130212381482
seconds:  0.0655538928695023
seconds:  0.06511583109386265
seconds:  0.060955140041187406
seconds:  0.06108091305941343
seconds:  0.061254137894138694
seconds:  0.06611824012361467
seconds:  0.061310650082305074



In [18]:
##### Network CPU

with tf.device('/CPU:0'):
    n_repeats = 10
    times = []
    for i in tqdm(range(n_repeats)):
        start_time = time.monotonic()
        embedder.transform(X_test_flat);
        end_time = time.monotonic()
        print('seconds: ', end_time - start_time)
        times.append(end_time - start_time)
        projection_speeds.loc[len(projection_speeds)] = ['network-cpu', 2, dataset, end_time - start_time]

,method_,dimensions,dataset,speed
0,network,2,macosko2015,0.152422
1,network,2,macosko2015,0.062610
2,network,2,macosko2015,0.061051
3,network,2,macosko2015,0.065554
4,network,2,macosko2015,0.065116
5,network,2,macosko2015,0.060955
6,network,2,macosko2015,0.061081
7,network,2,macosko2015,0.061254
8,network,2,macosko2015,0.066118
9,network,2,macosko2015,0.061311


##### 64 dims

In [19]:
load_loc = output_dir / dataset /"64"/ 'network' 

In [20]:
embedder = tfUMAP(
    direct_embedding=False,
    verbose=True,
    negative_sample_rate=5,
    training_epochs=5,
    batch_size = 100,
    dims = dims
)

In [21]:
encoder = tf.keras.models.load_model((load_loc / 'encoder').as_posix())
embedder.encoder = encoder

In [22]:
n_repeats = 10
times = []
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    embedder.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    times.append(end_time - start_time)
    projection_speeds.loc[len(projection_speeds)] = ['network', 64, dataset, end_time - start_time]

seconds:  0.15086891991086304
seconds:  0.07734775496646762
seconds:  0.07673786813393235
seconds:  0.08067622198723257
seconds:  0.08221768704243004
seconds:  0.08005523402243853
seconds:  0.07999808294698596
seconds:  0.07620651205070317
seconds:  0.0768652621190995
seconds:  0.07691968302242458



In [23]:
##### Network CPU

with tf.device('/CPU:0'):
    n_repeats = 10
    times = []
    for i in tqdm(range(n_repeats)):
        start_time = time.monotonic()
        embedder.transform(X_test_flat);
        end_time = time.monotonic()
        print('seconds: ', end_time - start_time)
        times.append(end_time - start_time)
        projection_speeds.loc[len(projection_speeds)] = ['network-cpu', 64, dataset, end_time - start_time]

,method_,dimensions,dataset,speed
0,network,2,macosko2015,0.152422
1,network,2,macosko2015,0.062610
2,network,2,macosko2015,0.061051
3,network,2,macosko2015,0.065554
4,network,2,macosko2015,0.065116
5,network,2,macosko2015,0.060955
6,network,2,macosko2015,0.061081
7,network,2,macosko2015,0.061254
8,network,2,macosko2015,0.066118
9,network,2,macosko2015,0.061311


#### UMAP-learn

##### 2 dims

In [24]:
embedder = UMAP(n_components = 2, verbose=True)
z_umap = embedder.fit_transform(X_train_flat)

UMAP(dens_frac=0.0, dens_lambda=0.0, verbose=True)
Construct fuzzy simplicial set
Wed Jul 15 15:37:51 2020 Finding Nearest Neighbors
Wed Jul 15 15:37:51 2020 Building RP forest with 14 trees
Wed Jul 15 15:37:52 2020 parallel NN descent for 15 iterations
	 0  /  15
	 1  /  15
	 2  /  15
	 3  /  15
	 4  /  15
	 5  /  15
Wed Jul 15 15:38:01 2020 Finished Nearest Neighbor Search
Wed Jul 15 15:38:04 2020 Construct embedding
	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Wed Jul 15 15:38:37 2020 Finished embedding


In [25]:
n_repeats = 10
times = []
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    embedder.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    times.append(end_time - start_time)
    projection_speeds.loc[len(projection_speeds)] = ['umap-learn', 2, dataset, end_time - start_time]


	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  15.734487792942673
	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  4.392539114924148
	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  3.900813676882535
	completed  0  /  

In [26]:
projection_speeds

,method_,dimensions,dataset,speed
0,network,2,macosko2015,0.152422
1,network,2,macosko2015,0.062610
2,network,2,macosko2015,0.061051
3,network,2,macosko2015,0.065554
4,network,2,macosko2015,0.065116
5,network,2,macosko2015,0.060955
6,network,2,macosko2015,0.061081
7,network,2,macosko2015,0.061254
8,network,2,macosko2015,0.066118
9,network,2,macosko2015,0.061311


##### 64 dims

In [27]:
embedder = UMAP(n_components = 64, verbose=True)
z_umap = embedder.fit_transform(X_train_flat)

UMAP(dens_frac=0.0, dens_lambda=0.0, n_components=64, verbose=True)
Construct fuzzy simplicial set
Wed Jul 15 15:39:29 2020 Finding Nearest Neighbors
Wed Jul 15 15:39:29 2020 Building RP forest with 14 trees
Wed Jul 15 15:39:29 2020 parallel NN descent for 15 iterations
	 0  /  15
	 1  /  15
	 2  /  15
	 3  /  15
	 4  /  15
	 5  /  15
Wed Jul 15 15:39:31 2020 Finished Nearest Neighbor Search
Wed Jul 15 15:39:31 2020 Construct embedding
	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Wed Jul 15 15:40:08 2020 Finished embedding


In [28]:
n_repeats = 10
times = []
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    embedder.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    projection_speeds.loc[len(projection_speeds)] = ['umap-learn', 64, dataset, end_time - start_time]


	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  5.335240128915757
	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  4.712620828999206
	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  4.208247289992869
	completed  0  /  1

In [29]:
projection_speeds

,method_,dimensions,dataset,speed
0,network,2,macosko2015,0.152422
1,network,2,macosko2015,0.062610
2,network,2,macosko2015,0.061051
3,network,2,macosko2015,0.065554
4,network,2,macosko2015,0.065116
5,network,2,macosko2015,0.060955
6,network,2,macosko2015,0.061081
7,network,2,macosko2015,0.061254
8,network,2,macosko2015,0.066118
9,network,2,macosko2015,0.061311


#### PCA

##### 2 dims

In [30]:
pca = PCA(n_components=2)
z = pca.fit_transform(X_train_flat)

In [31]:
n_repeats = 10
times = []
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    pca.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    times.append(end_time - start_time)
    projection_speeds.loc[len(projection_speeds)] = ['pca', 2, dataset, end_time - start_time]


seconds:  0.0010271200444549322
seconds:  0.0009383170399814844
seconds:  0.0009344569407403469
seconds:  0.0009289770387113094
seconds:  0.0009487872011959553
seconds:  0.0009352071210741997
seconds:  0.0009525769855827093
seconds:  0.0009377868846058846
seconds:  0.0009323679842054844
seconds:  0.0009314969647675753



In [32]:
projection_speeds

,method_,dimensions,dataset,speed
0,network,2,macosko2015,0.152422
1,network,2,macosko2015,0.062610
2,network,2,macosko2015,0.061051
3,network,2,macosko2015,0.065554
4,network,2,macosko2015,0.065116
5,network,2,macosko2015,0.060955
6,network,2,macosko2015,0.061081
7,network,2,macosko2015,0.061254
8,network,2,macosko2015,0.066118
9,network,2,macosko2015,0.061311


##### 64 dims

In [33]:
x_train_flat_padded = np.concatenate([X_train_flat, np.zeros((len(X_train_flat), 14))], axis=1)
X_test_flat_padded = np.concatenate([X_test_flat, np.zeros((len(X_test_flat), 14))], axis=1)

In [34]:
pca = PCA(n_components=64)
z = pca.fit_transform(x_train_flat_padded)

In [35]:
n_repeats = 10
times = []
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    pca.transform(X_test_flat_padded);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    times.append(end_time - start_time)
    projection_speeds.loc[len(projection_speeds)] = ['pca', 64, dataset, end_time - start_time]


seconds:  0.001996397040784359
seconds:  0.0022262651473283768
seconds:  0.0024269099812954664
seconds:  0.0023763689678162336
seconds:  0.002365178894251585
seconds:  0.0024054499808698893
seconds:  0.0023558679968118668
seconds:  0.002367038046941161
seconds:  0.0023865289986133575
seconds:  0.002360638929530978



In [36]:
projection_speeds

,method_,dimensions,dataset,speed
0,network,2,macosko2015,0.152422
1,network,2,macosko2015,0.062610
2,network,2,macosko2015,0.061051
3,network,2,macosko2015,0.065554
4,network,2,macosko2015,0.065116
5,network,2,macosko2015,0.060955
6,network,2,macosko2015,0.061081
7,network,2,macosko2015,0.061254
8,network,2,macosko2015,0.066118
9,network,2,macosko2015,0.061311


#### TSNE

##### 2 dims

In [37]:
tsne = TSNE(
    n_components = 2,
    n_jobs=32,
    verbose=True
)

In [38]:
embedding_train = tsne.fit(X_train_flat)

--------------------------------------------------------------------------------
TSNE(n_jobs=32, neighbors=None, verbose=True)
--------------------------------------------------------------------------------
===> Finding 90 nearest neighbors using Annoy approximate search using euclidean distance...


/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


   --> Time elapsed: 9.73 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.32 seconds
===> Calculating PCA-based initialization...
   --> Time elapsed: 0.08 seconds
===> Running optimization with exaggeration=12.00, lr=2527.25 for 250 iterations...
Iteration   50, KL divergence 5.8146, 50 iterations in 1.2758 sec
Iteration  100, KL divergence 5.2446, 50 iterations in 1.3230 sec
Iteration  150, KL divergence 5.1509, 50 iterations in 1.3168 sec
Iteration  200, KL divergence 5.1122, 50 iterations in 1.3505 sec
Iteration  250, KL divergence 5.0914, 50 iterations in 1.3720 sec
   --> Time elapsed: 6.64 seconds
===> Running optimization with exaggeration=1.00, lr=2527.25 for 500 iterations...
Iteration   50, KL divergence 3.5878, 50 iterations in 1.4364 sec
Iteration  100, KL divergence 3.1797, 50 iterations in 1.3553 sec
Iteration  150, KL divergence 2.9765, 50 iterations in 1.5623 sec
Iteration  200, KL divergence 2.8506, 50 iterations in 2.1569 sec
Iteration  250, KL div

In [39]:
n_repeats = 10
times = []
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    embedding_train.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    times.append(end_time - start_time)
    projection_speeds.loc[len(projection_speeds)] = ['TSNE', 2, dataset, end_time - start_time]

===> Finding 15 nearest neighbors in existing embedding using Annoy approximate search...
   --> Time elapsed: 0.85 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.02 seconds
===> Running optimization with exaggeration=4.00, lr=0.10 for 0 iterations...
   --> Time elapsed: 0.00 seconds
===> Running optimization with exaggeration=1.50, lr=0.10 for 250 iterations...
Iteration   50, KL divergence 83573.3050, 50 iterations in 0.1257 sec
Iteration  100, KL divergence 83673.9513, 50 iterations in 0.1224 sec
Iteration  150, KL divergence 83714.9859, 50 iterations in 0.1321 sec
Iteration  200, KL divergence 83746.5617, 50 iterations in 0.1257 sec
Iteration  250, KL divergence 83765.2234, 50 iterations in 0.1300 sec
   --> Time elapsed: 0.64 seconds
seconds:  2.302478270838037
===> Finding 15 nearest neighbors in existing embedding using Annoy approximate search...
   --> Time elapsed: 0.98 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.02 seconds
===> Run

In [40]:
projection_speeds

,method_,dimensions,dataset,speed
0,network,2,macosko2015,0.152422
1,network,2,macosko2015,0.062610
2,network,2,macosko2015,0.061051
3,network,2,macosko2015,0.065554
4,network,2,macosko2015,0.065116
...,...,...,...,...
65,TSNE,2,macosko2015,1.605844
66,TSNE,2,macosko2015,1.692429
67,TSNE,2,macosko2015,1.714449
68,TSNE,2,macosko2015,1.649606


### Save

In [41]:
save_loc = DATA_DIR / 'projection_speeds' / (dataset + '.pickle')
ensure_dir(save_loc)
projection_speeds.to_pickle(save_loc)